# Clinical Analysis: Prostate Tumor Stage at T2b

## Loading the Dataset file

In [ ]:
import pandas as pd
import numpy as np 
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")
#list(data.columns)
list(data["TUMOR_STAGE_T2b"].unique())

## Test for Normal Distribution and Homogeneity of Variance

In [ ]:
################## QQ plots with Shapiro Wilk Test Annotations:

# Load the data
import pandas as pd
import numpy as np 
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")

# Define function to plot QQ plots with Shapiro Wilk Test Annotations
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats

def analyze_gene(data, gene_name, hue_by, condition_order):
    sns.set(style="whitegrid")
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))
    axes = axes.flatten()
    for idx, condition in enumerate(condition_order):
        ax = axes[idx]
        subset_data = data[data[hue_by] == condition]
        # Check if the subset data has at least 3 data points
        if len(subset_data[gene_name]) < 3:
            ax.set_title(f'Not enough data for {gene_name}\n{condition}')
            continue
        sm.qqplot(subset_data[gene_name], line='s', ax=ax)
        ax.set_title(f'QQ Plot\n{condition}\n{gene_name}')
        # Perform Shapiro-Wilk test
        shapiro_test_stat, shapiro_p_value = stats.shapiro(subset_data[gene_name])
        color = 'red' if shapiro_p_value < 0.05 else 'green'
        ax.text(0.5, 0.1, f'S-W p-value: {shapiro_p_value:.4f}', transform=ax.transAxes, fontsize=10, color=color)
    plt.tight_layout()
    plt.savefig(f'{gene_name}_QQPLOT.png', format='png', dpi=600, bbox_inches='tight')
    plt.show()

# Specify the genes you want to analyze
genes_to_analyze = ['MIR17_EXP', 'MIR20B_EXP', 'MIR106A_EXP']  # Add more genes as needed
#Specify the phenotype you want to analyze
hue_by = 'TUMOR_STAGE_T2b_CONDITION'
# Specify the order of sample conditions
condition_order = ['PRE-RT, ≤T2b', '4WKSPD1, ≤T2b', '8WKSPD1, ≤T2b', 
                   'PRE-RT, >T2b', '4WKSPD1, >T2b', '8WKSPD1, >T2b']
# Call the function for each gene
for gene in genes_to_analyze:
    analyze_gene(data, gene, hue_by, condition_order)

###############################################################

In [ ]:
###################### Levene Test for Homogeneity of Variance:

import pandas as pd
import numpy as np
from scipy.stats import levene

def run_levene_test(dataframe, gene_to_analyze, groups_column, timepoints_column):
    timepoints = np.unique(dataframe[timepoints_column])
    groups = np.unique(dataframe[groups_column])
    data_levene = []
    for tp in timepoints:
        for group in groups:
            subset = dataframe[(dataframe[timepoints_column] == tp) & (dataframe[groups_column] == group)][gene_to_analyze].values
            data_levene.append(subset)
    statistic, p_values = levene(*data_levene)
    print(f'Test Statistics: {round(statistic, 3)}')
    print(f'p-values: {round(p_values, 3)}')

data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")
mirnas = ["MIR17_EXP", "MIR20B_EXP", "MIR106A_EXP"]

for mirna in mirnas:
    print(f'Levene Test for {mirna}')
    run_levene_test(dataframe = data, gene_to_analyze = mirna, groups_column = "TUMOR_STAGE_T2b", timepoints_column = "TIME_POINT")
    print("\n")
    
###############################################################

## Descriptive Statistics for the Gene Expression

In [ ]:
################### Obtaining Descr. Stat. on Clinical Outcome:

# Load the data
import pandas as pd
import numpy as np 
from datetime import datetime

data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")
current_date = datetime.now().strftime('%Y_%m_%d')
column = "TUMOR_STAGE_T2b"

# Perform descriptive statistics on the data frame
miRNA_exp_descr_stats = data.groupby(['TIME_POINT', column], sort=False)[['MIR17_EXP', 'MIR20B_EXP', 'MIR106A_EXP']].describe()
# Round the values to a specific number of decimals
miRNA_exp_descr_stats = miRNA_exp_descr_stats.round(decimals=3)
# Reset index to make the DataFrame more readable in the output
miRNA_exp_descr_stats.reset_index(inplace=True)
outputfile_name = f'{current_date}_{column}_DESC_STATS.xlsx'
# Save the result to a CSV file
miRNA_exp_descr_stats.to_excel(outputfile_name, index=True)
### Download and format in excel independently

###############################################################

## Running Mann-Whitney U Test Manually 

In [ ]:
################################### Mann Whitney U Test Results

import pandas as pd
import numpy as np 
from scipy.stats import mannwhitneyu

# Import the Data
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")

genes = ["MIR17_EXP", "MIR20B_EXP", "MIR106A_EXP"]
timepoints = ["PRE-RT", "4WKSPD1", "8WKSPD1"]
column1 = "TIME_POINT"
column2 = "TUMOR_STAGE_T2b"
groups = data[column2].unique()

# Subset the data for Mann-Whitney U Testing
tuple_mir17 = []
tuple_mir20b = []
tuple_mir106a = []
        
for gene in genes:    
    for tp in timepoints:
        subset1 = data[(data[column1] == f'{tp}') & (data[column2] == f'{groups[0]}')][gene].values
        subset2 = data[(data[column1] == f'{tp}') & (data[column2] == f'{groups[1]}')][gene].values
        mypair = (subset1, subset2)
        if gene == "MIR17_EXP":
            tuple_mir17.append(mypair)
        elif gene == "MIR20B_EXP":
            tuple_mir20b.append(mypair)
        else:
            tuple_mir106a.append(mypair)

### Accessing the Tuple
# tuple_miRNA[0-2], provides the timepoints, 0=prert, 1=4wkspd1, 2=8wkspd1
# tuple_miRNA[0-2][0-1], provides the group [0='≤T2b', 1='>T2b']

# Perform the Mann-Whitney U Test:
for miRNA in genes:
    print(f'Mann-Whitney U Test for {miRNA}')
    if miRNA == "MIR17_EXP":
        for idx, (group1_data, group2_data) in enumerate(tuple_mir17, start = 1):
            if idx == 1:
                print(f'Comparison at PRE-PT')
            elif idx == 2:
                print(f'Comparison at 4WKSPD1')
            else: 
                print(f'Comparison at 8WKSPD1')
            mannwhit_stat, p_value = mannwhitneyu(group1_data, group2_data, alternative = "two-sided")
            print(f"Mann-Whitney U Statistic: {mannwhit_stat:.3f}")
            print(f"P-value: {p_value:.3f}")
            if p_value < 0.05:
                print(f'Gene Expression is DIFFERENT')
            else:
                print(f'Gene Expression is NOT different')
            print("\n")
        print("\n"*3)
    elif miRNA == "MIR20B_EXP":
        for idx, (group1_data, group2_data) in enumerate(tuple_mir20b, start = 1):
            if idx == 1:
                print(f'Comparison at PRE-PT')
            elif idx == 2:
                print(f'Comparison at 4WKSPD1')
            else: 
                print(f'Comparison at 8WKSPD1')
            mannwhit_stat, p_value = mannwhitneyu(group1_data, group2_data, alternative = "two-sided")
            print(f"Mann-Whitney U Statistic: {mannwhit_stat:.3f}")
            print(f"P-value: {p_value:.3f}")
            if p_value < 0.05:
                print(f'Gene Expression is DIFFERENT')
            else:
                print(f'Gene Expression is NOT different')
            print("\n")
        print("\n"*3)
    else:
        for idx, (group1_data, group2_data) in enumerate(tuple_mir106a, start = 1):
            if idx == 1:
                print(f'Comparison at PRE-PT')
            elif idx == 2:
                print(f'Comparison at 4WKSPD1')
            else: 
                print(f'Comparison at 8WKSPD1')
            mannwhit_stat, p_value = mannwhitneyu(group1_data, group2_data, alternative = "two-sided")
            print(f"Mann-Whitney U Statistic: {mannwhit_stat:.3f}")
            print(f"P-value: {p_value:.3f}")
            if p_value < 0.05:
                print(f'Gene Expression is DIFFERENT')
            else:
                print(f'Gene Expression is NOT different')
            print("\n")
    
###############################################################

## Plots stratified according to Tumor Stage Group with Mann-Whitney U Test Results Annotations

In [ ]:
#################################### Mann-Whitney U Test Plots:

# Import Libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from statannotations.Annotator import Annotator

# Define function to plot Mann-Whitney U Test Plots
def plot_gene(x, data, hue, gene_name, ax, hue_order, custom_palette, PROPS, flierprops, stat_pairs):
    sns.set(style="whitegrid")
    # Plot Boxplot
    sns.boxplot(x=x, y=gene_name, data=data, order=["PRE-RT", "4WKSPD1", "8WKSPD1"], hue=hue, hue_order=hue_order,
                palette=custom_palette, showmeans=False, showfliers=False,
                meanprops={"marker": "s", "markerfacecolor": "black", "markersize": 5, "markeredgecolor": "black"},
                linewidth=1.5, medianprops=dict(color="black"), flierprops=flierprops, **PROPS, ax=ax)

    # Get the legend from just the box plot
    handles, labels = ax.get_legend_handles_labels()

    sns.swarmplot(x=x, y=gene_name, data=data, order=["PRE-RT", "4WKSPD1", "8WKSPD1"], hue=hue, hue_order=hue_order,
                  palette=custom_palette, dodge=True, size=5, linewidth=0.5, edgecolor="black", ax=ax)

    # Add Annotations
    annotator = Annotator(ax=ax, pairs=stat_pairs, data=data, x=x, y=gene_name, order=["PRE-RT", "4WKSPD1", "8WKSPD1"],
                          hue=hue, hue_order=hue_order, palette=custom_palette)
    
    annotator.configure(test="Mann-Whitney").apply_and_annotate() # comparisons_correction="Bonferroni"

    # Remove the old legend
    ax.legend_.remove()

    # Add just the handles/labels from the box plot back
    legend = ax.legend(handles, labels, loc=0)
    legend.set_title("PCa Tumor Stage", prop={"size": 9, "family": "serif"})
    for text in legend.texts:
        text.set_fontsize(9)  # Set the desired font size
        text.set_family("serif")  # Set the desired font family

    # Map and update the labels to new ones
    new_labels = {'≤T2b': "≤T2b", '>T2b': ">T2b"}
    for text, handle in zip(legend.texts, legend.legendHandles):
        text.set_text(new_labels[text.get_text()])

    labels = [item.get_text() for item in ax.get_xticklabels()]
    labels[0] = 'PRE-PT'
    labels[1] = '4WKSPD1'
    labels[2] = '8WKSPD1'
    ax.set_xticklabels(labels, family="serif")
    ax.set_yticklabels(ax.get_yticklabels(), family="serif")

    ax.xaxis.tick_top()
    ax.xaxis.tick_bottom()
    ax.tick_params(axis='x', labelsize=9, labelrotation=0)
    ax.tick_params(axis='y', labelsize=9)

    ax.set_xlabel('Time Point', family="serif", fontsize=12)
    ax.set_ylabel('Gene Expression Fold Change', family="serif", fontsize=12)
    variable = gene_name
    result = variable.split('_')[0]
    ax.set_title(result, family="serif")

###############################################################

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from statannotations.Annotator import Annotator

# Call the function to perform the plots 
# Load the data
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")
# Define the Pairs needed for the plot annotations with STATANNOTATOR
mannwhitu_testpairs = [[("PRE-RT", "≤T2b"), ("PRE-RT", ">T2b")],
                       [("4WKSPD1", "≤T2b"), ("4WKSPD1", ">T2b")],
                       [("8WKSPD1", "≤T2b"), ("8WKSPD1", ">T2b")]]

# Call Plotting Parameters
column1 = "TIME_POINT"
column2 = "TUMOR_STAGE_T2b"
group_hue_order = ["≤T2b", ">T2b"]
group_custom_palette = {"≤T2b" : "#E8E8E8", ">T2b": "#505050"}
group_PROPS = {'boxprops': {'edgecolor': 'black'}, 'whiskerprops': {'color': 'black'}, 'capprops': {'color': 'black'}}
group_flierprops = dict(markerfacecolor='grey', markersize=5, markeredgecolor='black')

# Begin Plotting
sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(30, 8), dpi=600)
# Call the function for each gene
genes_to_plot = ['MIR17_EXP', 'MIR20B_EXP', 'MIR106A_EXP']  # Add more genes as needed
for idx, gene in enumerate(genes_to_plot):
    plot_gene(x=column1, data=data, gene_name=gene, ax=axs[idx], hue = column2, hue_order=group_hue_order,
              custom_palette=group_custom_palette, PROPS=group_PROPS, flierprops = group_flierprops, 
              stat_pairs=mannwhitu_testpairs)

# Save individual gene plots
for idx, gene in enumerate(genes_to_plot):
    extent = axs[idx].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    variable = gene
    gene_name = variable.split('_')[0]
    fig.savefig(f'{column2}_{gene_name}_subplot.png', format='png', dpi=600, bbox_inches=extent.expanded(1.18, 1.2))

# Save combined plot
plt.savefig(f'{column2}_plot.png', format='png', dpi=600, bbox_inches='tight')
plt.show()

## Plots for each Tumor Stage Group with Kruskal Wallis Test Results Annotations

In [ ]:
import pandas as pd 
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")
column = "TUMOR_STAGE_T2b"
group = data[column].unique()
# Check group[0] is the lower group one, otherwise change to group[1]
#group1 = data[data[column] == group[0]]
#group1.head()

In [ ]:
################################# Kruskal Wallis Plots function:

# Import Libraries
import pandas as pd
import numpy as np 
from statannotations.Annotator import Annotator
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

# Define the plot for Kruskal Wallis Annotations to Group 1
def plot_gene_group(data, gene_name, ax, group_num):
    # Subset data
    column1 = "TIME_POINT"
    column2 = "TUMOR_STAGE_T2b"
    group = data[column2].unique()
    if group_num == 1:
        group_analysis = data[data[column2] == group[0]]
        custom_palette = {"PRE-RT": "#E8E8E8", "4WKSPD1": "#E8E8E8", "8WKSPD1": "#E8E8E8"}
    elif group_num == 2:
        group_analysis = data[data[column2] == group[1]]
        custom_palette = {"PRE-RT": "#505050", "4WKSPD1": "#505050", "8WKSPD1": "#505050"}
    
    PROPS = {'boxprops': {'edgecolor': 'black'}, 'whiskerprops': {'color': 'black'}, 'capprops': {'color': 'black'}}
    flierprops = dict(markerfacecolor='grey', markersize=5, markeredgecolor='black')
    tp_order = ["PRE-RT", "4WKSPD1", "8WKSPD1"]
    
    # Define Kruskal Wallis Test pairs
    tps = list(["PRE-RT", "4WKSPD1", "8WKSPD1"])
    tp_pairs = list(combinations(tps, r=2))
    
    # Start Plotting
    sns.set(style="whitegrid")
    
    sns.boxplot(x=column1, y=gene_name, data=group_analysis, order=tp_order, palette=custom_palette, showmeans=False,
                showfliers=False, meanprops={"marker": "s", "markerfacecolor": "black", "markersize": 5, "markeredgecolor": "black"},
                linewidth=1.5, medianprops=dict(color="black"), flierprops=flierprops, **PROPS, ax=ax)

    sns.swarmplot(x=column1, y=gene_name, data=group_analysis, order=tp_order, palette=custom_palette, dodge=True,
                  size=5, linewidth=0.5, edgecolor="black", ax=ax)

    annotator = Annotator(ax=ax, pairs=tp_pairs, data=group_analysis, x=column1, y=gene_name, order=tp_order, palette=custom_palette)
    annotator.configure(test='Kruskal', verbose = True).apply_and_annotate() #comparisons_correction="Holm-Bonferroni"

    labels = [item.get_text() for item in ax.get_xticklabels()]
    labels[0] = 'PRE-PT'
    labels[1] = '4WKSPD1'
    labels[2] = '8WKSPD1'
    ax.set_xticklabels(labels, family="serif")
    ax.set_yticklabels(ax.get_yticklabels(), family="serif")

    ax.xaxis.tick_top()
    ax.xaxis.tick_bottom()
    ax.tick_params(axis='x', labelsize=9, labelrotation=0)
    ax.tick_params(axis='y', labelsize=9)

    ax.set_xlabel('Time Point', family="serif", fontsize=12)
    ax.set_ylabel('Gene Expression Fold Change', family="serif", fontsize=12)
    variable = gene_name
    result = variable.split('_')[0]
    ax.set_title(result, family="serif")

###############################################################

In [ ]:
############################################## Group 1 Plotting

# Import Libraries 
import pandas as pd
import numpy as np
from statannotations.Annotator import Annotator
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
# Load the data
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")

# Begin Plotting
sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(30, 8), dpi=600)
# Call the function for each gene
genes_to_plot = ['MIR17_EXP', 'MIR20B_EXP', 'MIR106A_EXP']  # Add more genes as needed
for idx, gene in enumerate(genes_to_plot):
    plot_gene_group(data=data, gene_name=gene, ax=axs[idx], group_num=1)

# Save individual gene plots
name_phenotype = "TUMORSTAGE_T2b"

#for idx, gene in enumerate(genes_to_plot):
    #extent = axs[idx].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    #variable = gene
    #gene_name = variable.split('_')[0]
    #fig.savefig(f'Less_{name_phenotype}_{gene_name}_subplot.png', format='png', dpi=600, bbox_inches=extent.expanded(1.18, 1.2))

# Save combined plot
#plt.savefig(f'Less_{name_phenotype}_plot.png', format='png', dpi=600, bbox_inches='tight')
plt.show()

###############################################################

In [ ]:
############################################## Group 2 Plotting

# Import Libraries 
import pandas as pd
import numpy as np
from statannotations.Annotator import Annotator
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

# Load the data
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")

# Begin Plotting
sns.set(style="whitegrid")
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(30, 8), dpi=600)
# Call the function for each gene
genes_to_plot = ['MIR17_EXP', 'MIR20B_EXP', 'MIR106A_EXP']  # Add more genes as needed
for idx, gene in enumerate(genes_to_plot):
    plot_gene_group(data=data, gene_name=gene, ax=axs[idx], group_num=2)

# Save individual gene plots
name_phenotype = "TUMORSTAGE_T2b"
#for idx, gene in enumerate(genes_to_plot):
    #extent = axs[idx].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    #variable = gene
    #gene_name = variable.split('_')[0]
    #fig.savefig(f'Greater_{name_phenotype}_{gene_name}_subplot.png', format='png', dpi=600, bbox_inches=extent.expanded(1.18, 1.2))

# Save combined plot
#plt.savefig(f'Greater_{name_phenotype}_plot.png', format='png', dpi=600, bbox_inches='tight')
plt.show()

###############################################################

## Running Kruskal Wallis Test Manually 

In [ ]:
######################################## Kruskal-Wallis Results

import pandas as pd
import numpy as np 
from scipy import stats 
from scikit_posthocs import posthoc_dunn 

# Define Kruskal Wallis Test 
def run_kruskal_group1(dataframe, miRNA_col, group_column, tp_col):
    groups = data[group_column].unique()
    group1_subset = data[data[group_column] == groups[0]]
    tp = np.unique(data[tp_col])
    subset_data = [group1_subset[group1_subset[tp_col] == t][miRNA_col] for t in tp]
    kruskwall, p_value = stats.kruskal(*subset_data)
    print(f"Kruskal-Wallis Test Result for {miRNA_col} {groups[0]}")
    print(f'Test Statistic: {round(kruskwall, 3)}')
    print(f'p-value: {round(p_value, 3)}')
    dunn_df = posthoc_dunn(group1_subset, val_col=miRNA_col, group_col=tp_col)
    print(f"Dunn's Test Result for {miRNA_col}, Group: {groups[0]}")
    print(f'{round(dunn_df, 4)}')


# Import the Data
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")

column1 = "TIME_POINT"
column2 = "TUMOR_STAGE_T2b"
genes = ["MIR17_EXP", "MIR20B_EXP", "MIR106A_EXP"]

for miRNA in genes:
    run_kruskal_group1(dataframe=data, miRNA_col=miRNA, group_column=column2, tp_col=column1)
    print("\n")

###############################################################

In [ ]:
######################################## Kruskal-Wallis Results

import pandas as pd
import numpy as np 
from scipy import stats 
from scikit_posthocs import posthoc_dunn 

# Define Kruskal Wallis Test 
def run_kruskal_group2(dataframe, miRNA_col, group_column, tp_col):
    groups = data[group_column].unique()
    group2_subset = data[data[group_column] == groups[1]]
    tp = np.unique(data[tp_col])
    subset_data = [group2_subset[group2_subset[tp_col] == t][miRNA_col] for t in tp]
    kruskwall, p_value = stats.kruskal(*subset_data)
    print(f"Kruskal-Wallis Test Result for {miRNA_col} {groups[1]}")
    print(f'Test Statistic: {round(kruskwall, 4)}')
    print(f'p-value: {round(p_value, 4)}')
    dunn_df = posthoc_dunn(group2_subset, val_col=miRNA_col, group_col=tp_col)
    print(f"Dunn's Test Result for {miRNA_col}, Group: {groups[1]}")
    print(f'{round(dunn_df, 4)}')


# Import the Data
data = pd.read_excel("/home/johnny.velasquez/OTP_RADGEN_CLINICALDATA_ANALYSIS/DATA_PREPROCESSING/OTP_RADIOGENOMICS_FINAL_CLINICALDATASET_DEC12_2023_V4.xlsx")

column1 = "TIME_POINT"
column2 = "TUMOR_STAGE_T2b"
genes = ["MIR17_EXP", "MIR20B_EXP", "MIR106A_EXP"]

for miRNA in genes:
    run_kruskal_group2(dataframe=data, miRNA_col=miRNA, group_column=column2, tp_col=column1)
    print("\n")

###############################################################